In [19]:
import os
import praw
import pandas as pd
import numpy as np

import csv
import pandas as pd
import json
import pickle

import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline
reasult = pd.DataFrame()

In [2]:
from prawcore.exceptions import Forbidden

df = pd.DataFrame()

def scrapping(user, own_limit):
    reasult = pd.DataFrame()
    try:
        for submission in reddit.subreddit(user).hot(limit=own_limit):
            reasult = reasult.append({'user': user, 'text': submission.selftext}, ignore_index=True)
    except Forbidden:
        print('This is forbidden')
    return reasult

subreddits = ['DankMemes', 'Imgoingtohellforthis', 'KotakuInAction', 'MensRights', 'MetaCanada', 'MGTOW', 'PussyPass', 'PussyPassDenied', 'Donald', 'TumblrInAction']
for subreddit in subreddits:
    df = df.append(scrapping(subreddit, 200))

This is forbidden
This is forbidden
This is forbidden


In [5]:
df.to_csv('hsd/Reddit/validate_reddit.csv')

In [14]:
 df['text'].replace('', np.nan, inplace=True)

In [15]:
df.dropna(subset=['text'], inplace=True)

In [18]:
len(df)

162

In [20]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

In [23]:
df['lemmatized_text'] = df['text'].apply(lambda words: preprocess(words)[0])

In [24]:
df.to_csv('hsd/Reddit/validate_reddit.csv')